In [1]:
import pandas as pd
df = pd.read_csv('./data/SimLex-999.txt', delimiter = "\t")

In [2]:
df

,word1,word2,POS,SimLex999,conc(w1),conc(w2),concQ,Assoc(USF),SimAssoc333,SD(SimLex)
0,old,new,A,1.58,2.72,2.81,2,7.25,1,0.41
1,smart,intelligent,A,9.20,1.75,2.46,1,7.11,1,0.67
2,hard,difficult,A,8.77,3.76,2.21,2,5.94,1,1.19
3,happy,cheerful,A,9.55,2.56,2.34,1,5.85,1,2.18
4,hard,easy,A,0.95,3.76,2.07,2,5.82,1,0.93
...,...,...,...,...,...,...,...,...,...,...
994,join,acquire,V,2.85,2.86,2.93,2,0.00,0,0.99
995,send,attend,V,1.67,2.70,3.17,2,0.00,0,1.44
996,gather,attend,V,4.80,2.75,3.17,2,0.00,0,1.97
997,absorb,withdraw,V,2.97,3.11,3.04,2,0.00,0,1.75


In [3]:
# mm_model_path = 'multimodal_lstm_200000_1.0_15v6'#'multimodal_lstm_200000_1.0_15v5'##'multimodal_lstm_200000_0.7071067811865475_15v2'
# bm_model_path = 'benchmark_model_200000_1.0_15v7'#benchmark_model_200000_1.0_15v6'#'benchmark_model_200000_0.7071067811865475_15v4'

In [4]:
mm_model_path = "finetuned_multimodal_lstm"#'multimodal_lstm_200000_1.0_15v6'#'multimodal_lstm_200000_1.0_15v5'##'multimodal_lstm_200000_0.7071067811865475_15v2'
bm_model_path = 'finetuned_benchmark_lstm'#benchmark_model_200000_1.0_15v6'#'benchmark_model_200000_0.7071067811865475_15v4'

In [5]:
import torch
device = 'cuda' if torch.cuda.is_available() else "gpu"
from multimodal_model import BenchmarkLSTM, MMLSTM
from multi_bpe import MultiBPE
multi_bpe = MultiBPE()
def load_model(loaded_model, is_benchmark, is_multimodal, train_visual_module):
    if is_benchmark:
        mm_model = BenchmarkLSTM().to(device)
    else:
        mm_model = MMLSTM(is_multimodal=is_multimodal,
                     train_visual_module=train_visual_module).to(device)
    
    mm_model.load_state_dict(torch.load(f'./saved_models/{loaded_model}'))
    mm_model.eval()
    return mm_model

mm_model = load_model(mm_model_path, False, True, False)
bm_model = load_model(bm_model_path, True, False, False)

In [6]:
def cosine_score(w1, w2, model, name='emb_layer.weight'):
    tcs = torch.nn.CosineSimilarity(dim=1)
    e1 = multi_bpe.encode(w1, False, False, None)
    e2 = multi_bpe.encode(w2, False, False, None)
    if len(e1) > 1 or len(e2) > 1:
        a = model.state_dict()[name][e1[0]]
        for idx in e1[1:]:
            a = torch.concat((a, model.state_dict()[name][idx]))
        b = model.state_dict()[name][e2[0]]
        for idx in e2[1:]:
            b = torch.concat((b, model.state_dict()[name][idx]))
        return 999
        return tcs(torch.tensor(a).view(1,-1),torch.tensor(b).view(1,-1)).item()
    else:
        a = model.state_dict()[name][e1[0]]
        b = model.state_dict()[name][e2[0]]
        return tcs(torch.tensor(a).view(1,-1),torch.tensor(b).view(1,-1)).item()

def cosine_sim_model(df, model, name):
    l = []    
    for i in range(len(df)):
        w1, w2 = df.word1[i], df.word2[i]
        l.append(cosine_score(w1, w2, model, name))
    return l

In [7]:
df['bm_sim'] = cosine_sim_model(df, bm_model, 'fc.weight')
df['mm_sim'] = cosine_sim_model(df, mm_model, 'fc.weight')

<ipython-input-6-b642a3c19c31>:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return tcs(torch.tensor(a).view(1,-1),torch.tensor(b).view(1,-1)).item()
<ipython-input-6-b642a3c19c31>:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return tcs(torch.tensor(a).view(1,-1),torch.tensor(b).view(1,-1)).item()


In [8]:
df = df[df.bm_sim != 999].reset_index(drop=True)
df

,word1,word2,POS,SimLex999,conc(w1),conc(w2),concQ,Assoc(USF),SimAssoc333,SD(SimLex),bm_sim,mm_sim
0,old,new,A,1.58,2.72,2.81,2,7.25,1,0.41,0.813116,0.797752
1,smart,intelligent,A,9.20,1.75,2.46,1,7.11,1,0.67,0.866949,0.889700
2,hard,difficult,A,8.77,3.76,2.21,2,5.94,1,1.19,0.873111,0.878211
3,hard,easy,A,0.95,3.76,2.07,2,5.82,1,0.93,0.882229,0.849717
4,fast,rapid,A,8.75,3.32,3.07,2,5.66,1,1.68,0.826495,0.820413
...,...,...,...,...,...,...,...,...,...,...,...,...
726,borrow,want,V,1.77,3.10,1.93,1,0.00,0,1.22,0.816894,0.846424
727,absorb,possess,V,5.00,3.11,2.60,2,0.00,0,1.15,0.999762,0.999650
728,send,attend,V,1.67,2.70,3.17,2,0.00,0,1.44,0.810286,0.849188
729,gather,attend,V,4.80,2.75,3.17,2,0.00,0,1.97,0.890487,0.872842


In [9]:
df.corr(method='pearson')

,SimLex999,conc(w1),conc(w2),concQ,Assoc(USF),SimAssoc333,SD(SimLex),bm_sim,mm_sim
SimLex999,1.000000,-0.112686,-0.091588,-0.114723,0.194707,0.258558,-0.044860,0.102719,0.083347
conc(w1),-0.112686,1.000000,0.771252,0.892806,0.090763,0.105300,0.020184,0.023176,0.020497
conc(w2),-0.091588,0.771252,1.000000,0.903292,0.080963,0.070307,0.011213,0.031880,0.038481
concQ,-0.114723,0.892806,0.903292,1.000000,0.082716,0.084667,-0.006772,0.003623,0.011162
Assoc(USF),0.194707,0.090763,0.080963,0.082716,1.000000,0.608795,-0.152883,-0.009182,0.013634
SimAssoc333,0.258558,0.105300,0.070307,0.084667,0.608795,1.000000,-0.117750,-0.021776,-0.015188
SD(SimLex),-0.044860,0.020184,0.011213,-0.006772,-0.152883,-0.117750,1.000000,-0.089840,-0.104586
bm_sim,0.102719,0.023176,0.031880,0.003623,-0.009182,-0.021776,-0.089840,1.000000,0.876862
mm_sim,0.083347,0.020497,0.038481,0.011162,0.013634,-0.015188,-0.104586,0.876862,1.000000


In [10]:
import pingouin as pg

#find partial correlation between hours and exam score while controlling for grade
pg.partial_corr(data=df, x='mm_sim', y='SimLex999', covar='bm_sim')

,n,r,CI95%,p-val
pearson,731,-0.014059,"[-0.09, 0.06]",0.704529


In [11]:
0.445846**2

0.198778655716

In [12]:
-0.081773**2

-0.006686823529

In [13]:
0.041998**2

0.001763832004

In [14]:
0.124926**2

0.015606505476